**Import dependecies**

In [ ]:
'''
%conda install -c apple tensorflow-deps scipy #macos
%pip install tensorflow-macos tensorflow-metal #macos

%pip install --upgrade tensorflow 
%pip install pydot graphviz

%pip install pandas numpy pillow matplotlib scipy autopep8 pydot
'''

In [ ]:
import tensorflow as tf
%load_ext tensorboard

In [ ]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.config.list_physical_devices()

In [ ]:
import os
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
import datetime

fruit = "apple"
model_current = "vgg16"
curr_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_prefix = f'./runs/{fruit}/{model_current}/{curr_datetime}/'
#results_prefix = f'./results/{fruit}/{model_current}/{curr_datetime}/'
log_dir = f'logs/fit/{fruit}/{model_current}/{curr_datetime}'
os.makedirs(run_prefix, exist_ok=True)
#os.makedirs(results_prefix, exist_ok=True)
run_prefix


**Create an imagedatagenerator object to label data**

In [ ]:
input_size = 224
batch_size = 16
train_size = 8000
test_size = 2000
learning_rate = 0.001
epochs = 25
dropout = 0.5
dense_size = 1024

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_dir = f"./dataset_frutas_colab/{fruit}/train"
test_dir = f"./dataset_frutas_colab/{fruit}/test"
train_generator = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,class_mode = 'categorical', shuffle= True, target_size=(input_size,input_size), seed=42)
test_generator = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,class_mode = 'categorical',shuffle= False, target_size=(input_size,input_size))
train_generator.class_indices

VGG16 model

In [ ]:
pre_trained_model = VGG16(
    input_shape = (input_size,input_size,3),
    include_top = False,
    weights='imagenet'
)


Start transfer learning process

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
#Flatten
x = layers.Flatten()(pre_trained_model.output)
#Fully connected layer con 1,024 hidden units y ReLU
x = layers.Dense(dense_size, activation='relu')(x)
x = layers.Dense(dense_size/2, activation='relu')(x)
#Dropout rate
if (dropout>0):
    x = layers.Dropout(dropout)(x)
#Binary classification and softmax
x = layers.Dense(2,activation='softmax')(x)

model = Model(pre_trained_model.input,x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics = [tf.keras.metrics.Accuracy(),
                         tf.keras.metrics.Precision(), 
                         tf.keras.metrics.Recall()])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file=f"{run_prefix}model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)
print("Saved Model Image")

In [ ]:
import datetime

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(
    train_generator,
    validation_data = test_generator,
    steps_per_epoch = train_size/batch_size,
    epochs = epochs,
    validation_steps = test_size/batch_size,
    callbacks=[tensorboard_callback]
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
precision = history.history['precision']
recall = history.history['recall']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.savefig(f"{run_prefix}acc.svg")
plt.figure()


plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig(f"{run_prefix}loss.svg")

plt.show()

In [ ]:
model.save(f"{run_prefix}{model_current}.h5")

In [ ]:
import csv

header = ["train_acc", "val_acc", "train_loss", "val_loss", "precision", "recall"]

with open(f'{run_prefix}{epochs}-{dropout}-{batch_size}-{learning_rate}-{input_size}-RMS-metrics.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    for idx in range(len(acc)):
        line = [acc[idx],val_acc[idx],loss[idx],val_loss[idx], precision[idx], recall[idx]]        
        writer.writerow(line)


Probar la prediccion

In [ ]:
# TensorFlow and tf.keras
from keras.preprocessing import image
import numpy as np
# Helper libraries

fresh_dir = './test_apple/fresh'
defect_dir = './test_apple/defect'

fresh_files = os.listdir(fresh_dir)
defect_files = os.listdir(defect_dir)

#model = keras.models.load_model('mobilenetv2_16_30_0.2.h5')

count_fresh_right = 0
count_fresh_wrong = 0
count_defect_right = 0
count_defect_wrong = 0

fresh_prob = []
defect_prob = []

for file in fresh_files:
    img = tf.keras.utils.load_img(
        fresh_dir+'/'+file, target_size=(input_size, input_size))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    images = np.vstack([img_array])
    classes = model.predict(images, batch_size=batch_size)
    fresh_prob.append((classes[0][0], classes[0][1]))
    if classes[0][0] > 0.5:
        count_fresh_wrong += 1
    elif classes[0][1] > 0.5:
        count_fresh_right += 1


for file in defect_files:
    img = tf.keras.utils.load_img(
        defect_dir+'/'+file, target_size=(input_size, input_size))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    images = np.vstack([img_array])
    classes = model.predict(images, batch_size=batch_size)
    defect_prob.append((classes[0][0], classes[0][1]))
    if classes[0][0] > 0.5:
        count_defect_right += 1
    elif classes[0][1] > 0.5:
        count_defect_wrong += 1


print("Fresh right: ", count_fresh_right)
print("Fresh wrong: ", count_fresh_wrong)
print("Defect right: ", count_defect_right)
print("Defect wrong: ", count_defect_wrong)


with open(f'{run_prefix}{epochs}-{dropout}-{batch_size}-{learning_rate}-{input_size}-RMS-predicction.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header1
    writer.writerow(["fresh_right", "fresh_wrong",
                    "defect_right", "defect_wrong"])
    writer.writerow([count_fresh_right, count_fresh_wrong,
                    count_defect_right, count_defect_wrong])
    # write the header2
    writer.writerow(["fresh_negative", "fresh_positive",
                    "defect_positive", "defect_negative"])
    for idx in range(len(fresh_prob)):
        writer.writerow([fresh_prob[idx][0], fresh_prob[idx][1],
                        defect_prob[idx][0], defect_prob[idx][1]])
